In [84]:
from tone_normalization import normalize_vn_tone

df_train = df_train.with_columns(df_train["text"].map_elements(function=normalize_vn_tone, return_dtype=pl.String))

### Remove error files

In [85]:
remove_lines = open("/home/lmnguyen/Projects/cmc_stt/finetune-fast-conformer/notebooks/resources/results/final_text_remove.txt").read().split("\n")
len(remove_lines)

9280

In [86]:
df_train = df_train.filter(~pl.col("file").is_in(remove_lines))
df_train

file,text,duration
str,str,f64
"""/home/ntdong/D…","""nhưng đã không…",4.2873125
"""/home/ntdong/D…","""anh sẽ giữ dùm…",1.8
"""/home/ntdong/D…","""hai ca sĩ việt…",6.1300625
"""/home/ntdong/D…","""hiện tình trạn…",3.34
"""/home/ntdong/D…","""bà cấn thị thê…",10.83
…,…,…
"""/home/ntdong/D…","""cậu ơi tắt máy…",2.28
"""/home/ntdong/D…","""kiểm tra cho t…",2.2
"""/home/ntdong/D…","""em ơi đóng giú…",3.51


In [87]:
1_799_683 - 1_791_685 

7998

### Replace wrong word in datasets

In [88]:
error_correct_dict = {}
for line in open("/home/lmnguyen/Projects/cmc_stt/finetune-fast-conformer/notebooks/resources/results/final_filter_merge.txt").read().split("\n"):
    if line:
        try:
            error, correct = line.split("#")
        except:
            print(line)
        error_correct_dict[error.strip()] = correct.strip()

# display some samples
list(error_correct_dict.items())[:10], len(error_correct_dict)

([('aastrazeneca', 'astrazeneca'),
  ('abbank', 'true'),
  ('abivin', 'true'),
  ('ac', 'true'),
  ('acb', 'true'),
  ('acd', 'true'),
  ('acls', 'true'),
  ('acopan', 'false'),
  ('acv', 'true'),
  ('adarninnalyn', 'adrenalin')],
 1668)

In [ ]:
error_correct_dict["c m c"] = "cmc"


In [93]:
# deep copy df_train
df_train_copy = df_train.clone()

In [94]:
# there are 3 cases:
# 1. value is 'true' --> we don't need to do anything
# 2. value is 'false' --> we need to remove all the rows in dataframe that text contain the key
# 3. value is a string --> we need to replace all the text in dataframe that contain the key by the value
from tqdm import tqdm
for wrong, correct in tqdm(error_correct_dict.items()):
    if correct == "false":
        df_train_copy = df_train_copy.filter(~pl.col("text").str.contains(rf'\b{wrong}\b'))
        print(wrong, len(df_train_copy))

wrong_files = []
for wrong, correct in tqdm(error_correct_dict.items()):
    if correct in ["true", "false"]:
        continue
    
    wrong_files.extend(df_train_copy.filter(df_train_copy["text"].str.contains(rf'\b{wrong}\b'))["file"].to_list())
    df_train_copy = df_train_copy.with_columns(df_train_copy["text"].replace(rf'\b{wrong}\b', correct))
    print(wrong, len(df_train_copy))


  8%|▊         | 140/1668 [00:00<00:02, 625.77it/s]

acopan 1791676
agromet 1791668
aj tôi không 1791666
nó không có đủ ampe á em 1791663
biê 1791656
bn 1791651
bă 1791649


 16%|█▋        | 275/1668 [00:00<00:03, 367.06it/s]

ch 1791630
checkin 1791627
chiê 1791626
châ 1791623
cuô 1791616


 19%|█▉        | 315/1668 [00:00<00:04, 308.18it/s]

...

đđến 1791105
đĩđể 1791105
đơnchắc 1791105


100%|█████████▉| 1665/1668 [00:36<00:00, 20.16it/s]

đưacon 1791105
đươc 1791105
đưọc 1791105


100%|██████████| 1668/1668 [00:37<00:00, 45.08it/s]

đẹpmỗi 1791105
ảrập 1791105
ấylắc 1791105


In [96]:
len(set(wrong_files))

7758

In [97]:
7758 + 9280 + 550

17588

In [81]:
df_train

file,text,duration
str,str,f64
"""/home/ntdong/D…","""nhưng đã không…",4.2873125
"""/home/ntdong/D…","""anh sẽ giữ dùm…",1.8
"""/home/ntdong/D…","""hai ca sĩ việt…",6.1300625
"""/home/ntdong/D…","""hiện tình trạn…",3.34
"""/home/ntdong/D…","""bà cấn thị thê…",10.83
…,…,…
"""/home/ntdong/D…","""cậu ơi tắt máy…",2.28
"""/home/ntdong/D…","""kiểm tra cho t…",2.2
"""/home/ntdong/D…","""em ơi đóng giú…",3.51


In [101]:
df_train_copy.write_csv("/home/lmnguyen/Projects/cmc_stt/finetune-fast-conformer/notebooks/resources/results/final_correct_dataset_cmc_filter.csv")